<a href="https://colab.research.google.com/github/azdinelaaouissi/ML/blob/main/AVITO_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas  as  pd
import  numpy  as  np
import  seaborn as sns
import  matplotlib.pyplot as plt
import  plotly.express as  px
import  warnings
from  sklearn.model_selection import train_test_split
from  sklearn.linear_model  import LinearRegression
from  sklearn.preprocessing  import  OneHotEncoder,StandardScaler,LabelEncoder
from sklearn.metrics import mean_squared_error,accuracy_score,confusion_matrix, r2_score
warnings.filterwarnings("ignore")

In [45]:
df=pd.read_csv("avito_data_ALL.csv")


In [46]:
df.head()

,Unnamed: 0.1,Unnamed: 0,marque,modele,annee_modele,kilometrage,Puissance_fiscale,carburant,ville,etat,prix
0,0,0.0,Opel,Astra,2011,110 000 - 119 999,6 CV,Diesel,Fès,Excellent,120000 DHs
1,1,1.0,Hyundai,Accent,2023,30 000 - 34 999,6 CV,Diesel,Casablanca,Très bon,164000 DHs
2,2,2.0,Peugeot,301,2018,110 000 - 119 999,6 CV,Diesel,Casablanca,Excellent,102000 DHs
3,3,3.0,Peugeot,208,2018,80 000 - 84 999,6 CV,Essence,Casablanca,Excellent,87000 DHs
4,4,4.0,Porsche,Macan,2018,120 000 - 129 999,12 CV,Diesel,El Jadida,Excellent,NaN


In [47]:
df.drop(df.iloc[:,:2],axis=1,inplace=True)

In [48]:
df.isna().sum()

marque                  6
modele                  6
annee_modele            5
kilometrage             6
Puissance_fiscale     125
carburant               4
ville                   0
etat                 1358
prix                 5586
dtype: int64

In [49]:
df.dropna(inplace=True)

In [50]:
df.shape

(14278, 9)

In [51]:
df.duplicated().sum()

374

In [52]:
df.drop_duplicates(inplace=True)

In [53]:
df.shape

(13904, 9)

In [54]:
df.annee_modele=df.annee_modele.str.slice(0,4).astype(int)

In [55]:
df.drop(df[df['kilometrage']=="Plus de 500 000"].index,inplace=True)

In [56]:
df.shape

(13795, 9)

In [57]:
df['kilometrage'] = df['kilometrage'].str.split("-").str[0]
df['kilometrage'] = df['kilometrage'].str.replace(" ", "")
df['kilometrage'] = df['kilometrage'].astype(float).astype('Int64')

In [58]:
df["age"]=2024-df.annee_modele

In [59]:
df.drop("annee_modele",axis=1,inplace=True)

In [60]:
df["Puissance_fiscale"].replace("Plus de 41 CV","42",inplace=True)

In [61]:
df["Puissance_fiscale"]=df.Puissance_fiscale.str.split(' ').str[0].astype(float).astype('Int64')

In [62]:
df["prix"]=df.prix.str.split(" ").str[0].astype(float).astype('Int64')

In [63]:
low_interval = (df['kilometrage'].min(), df['kilometrage'].quantile(0.25))
medium_interval = (df['kilometrage'].quantile(0.25), df['kilometrage'].quantile(0.75))
high_interval = (df['kilometrage'].quantile(0.75), df['kilometrage'].max())

In [64]:
def categorize_kilometrage(km):
    if km <= low_interval[1]:
        return 'low'
    elif km <= medium_interval[1]:
        return 'medium'
    else:
        return 'high'

In [65]:
df['kilometrage_category'] = df['kilometrage'].apply(categorize_kilometrage)

In [66]:
Q1 = df['prix'].quantile(0.25)
Q3 = df['prix'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

114000.0


In [67]:
df = df[~((df['prix'] < (Q1 - 1.5 * IQR)) |(df['prix'] > (Q3 + 0.7 * IQR)))]
df.shape

(12058, 10)

In [68]:
df.head()

,marque,modele,kilometrage,Puissance_fiscale,carburant,ville,etat,prix,age,kilometrage_category
0,Opel,Astra,110000,6,Diesel,Fès,Excellent,120000,13,medium
1,Hyundai,Accent,30000,6,Diesel,Casablanca,Très bon,164000,1,low
2,Peugeot,301,110000,6,Diesel,Casablanca,Excellent,102000,6,medium
3,Peugeot,208,80000,6,Essence,Casablanca,Excellent,87000,6,medium
5,Mercedes-Benz,Classe SLK,110000,9,Essence,Marrakech,Excellent,235000,10,medium


In [69]:
# Liste des valeurs à supprimer
valeurs_a_supprimer = ["72", "73", "74", "75", "76", "77", "97"]

# Filtrer les lignes où la colonne 'marque' est dans la liste des valeurs à supprimer
df = df[~df["marque"].isin(valeurs_a_supprimer)]

In [70]:
df.shape

(12039, 10)

In [71]:
df.head()

,marque,modele,kilometrage,Puissance_fiscale,carburant,ville,etat,prix,age,kilometrage_category
0,Opel,Astra,110000,6,Diesel,Fès,Excellent,120000,13,medium
1,Hyundai,Accent,30000,6,Diesel,Casablanca,Très bon,164000,1,low
2,Peugeot,301,110000,6,Diesel,Casablanca,Excellent,102000,6,medium
3,Peugeot,208,80000,6,Essence,Casablanca,Excellent,87000,6,medium
5,Mercedes-Benz,Classe SLK,110000,9,Essence,Marrakech,Excellent,235000,10,medium


In [72]:
df2=df.copy()

In [73]:
l_encode=["carburant","etat","kilometrage_category"]
label_encoder = LabelEncoder()
for  col  in  l_encode:
    df[col]=label_encoder.fit_transform(df[col])

In [74]:
df.head()

,marque,modele,kilometrage,Puissance_fiscale,carburant,ville,etat,prix,age,kilometrage_category
0,Opel,Astra,110000,6,0,Fès,3,120000,13,2
1,Hyundai,Accent,30000,6,0,Casablanca,6,164000,1,1
2,Peugeot,301,110000,6,0,Casablanca,3,102000,6,2
3,Peugeot,208,80000,6,2,Casablanca,3,87000,6,2
5,Mercedes-Benz,Classe SLK,110000,9,2,Marrakech,3,235000,10,2


In [75]:
categorical_col=["modele","marque","ville"]
df= pd.get_dummies(df, columns=categorical_col)

In [ ]:
#import category_encoders as ce
#for  col  in  ["modele","marque","ville"] :
  #encoder = ce.TargetEncoder(cols=[col])
 # df[col] = encoder.fit_transform(df[col], df['prix'])
#df.head()

In [76]:
df.head()

,kilometrage,Puissance_fiscale,carburant,etat,prix,age,kilometrage_category,modele_104,modele_106,modele_107,...,ville_Zaïo,ville_Zeghanghane,ville_Zemamra,ville_Zenata,ville_Ziaida,ville_أكادير,ville_الدار البيضاء,ville_الرباط,ville_تطوان,ville_طنجة
0,110000,6,0,3,120000,13,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,30000,6,0,6,164000,1,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,110000,6,0,3,102000,6,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,80000,6,2,3,87000,6,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,110000,9,2,3,235000,10,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [77]:
X=df.drop(["prix"],axis=1)
y=df["prix"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [78]:

model=LinearRegression()

In [80]:
model.fit(X_train,y_train)
# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul du coefficient de détermination (R²)
r_squared = r2_score(y_test, y_pred)
print(f"Coefficient de détermination (R²): {r_squared}")

# Calcul du Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")


Coefficient de détermination (R²): 0.700136402735001
Mean Squared Error (MSE): 1082087410.3790429


In [81]:
model.score(X_train, y_train)

0.7551380941249186

In [82]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(max_depth = 5, n_jobs = -1)
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, n_jobs=-1)

In [83]:
y_pred = forest.predict(X_test)
# Faire des prédictions sur l'ensemble de test
y_pred = forest.predict(X_test)

# Calcul du coefficient de détermination R²
r2 = r2_score(y_test, y_pred)

# Affichage du coefficient de détermination R²
print("Coefficient de détermination R² :", r2)

Coefficient de détermination R² : 0.6487176256844327


In [84]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth = 5)
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5)

In [85]:
y_pred_tree = tree.predict(X_test)

# Calcul du coefficient de détermination R²
r2_tree = r2_score(y_test, y_pred_tree)

# Affichage du coefficient de détermination R²
print("Coefficient de détermination R² (arbre de décision) :", r2_tree)

Coefficient de détermination R² (arbre de décision) : 0.6291021975063658
